Prepare the data, you only need to run this line once

In [ ]:
from data import prepare_hdf5
prepare_hdf5()

In [ ]:
import pandas as pd
import cufflinks

In [ ]:
all_data = pd.read_hdf('data/data.h5', "24g/PM10")
metadata = pd.read_hdf('data/data.h5', "metadata")

In [ ]:
cols = all_data.columns.str.startswith("")
all_data.rolling(30).mean().iloc[:,cols].iplot()

In [ ]:
all_data.iloc[:,cols].groupby(pd.Grouper(freq="M")).median().iplot()

In [ ]:
import pandas_bokeh
pandas_bokeh.output_notebook()

In [ ]:
E_minmax = metadata['web_mercator_E'].min(), metadata['web_mercator_E'].max()
N_minmax = metadata['web_mercator_N'].min(), metadata['web_mercator_N'].max()

In [ ]:

summary_stats = all_data.groupby(pd.Grouper(freq="M")).mean()
dates = summary_stats.index #.strftime('%Y-%M').values
data = {}
for date in dates:
    key = date.strftime("%Y-%M")
    df = metadata.copy().set_index('Kod stacji')
    df['summary'] = summary_stats.loc[dates[0]].T
    data[key] = ColumnDataSource(df)

In [ ]:
from bokeh.layouts import row, column, grid
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider
from bokeh.models import HoverTool, PanTool, WheelZoomTool, DateRangeSlider
from bokeh.models import ColumnDataSource

source = ColumnDataSource(metadata)

# def slider_update(attrname, old, new):
#     year = slider.value
#     label.text = str(year)
#     source.data = data[year]

# slider = DateRangeSlider(start=dates[0].date(), end=dates[-1].date(), title="Year-Month")
# slider.on_change('value', slider_update)

hover = HoverTool(
        tooltips=[
            ("Stacja", "@{Nazwa stacji}")
        ]
    )

fig = figure(
    tools=[hover, PanTool(), WheelZoomTool()],
    x_range=E_minmax,
    y_range=N_minmax,
    x_axis_type="mercator",
    y_axis_type="mercator")
fig.add_tile(get_provider('CARTODBPOSITRON_RETINA'))
fig.circle(
    x='web_mercator_E',
    y='web_mercator_N',
    source=ColumnDataSource(metadata),
    line_color='grey',
    fill_color='yellow')

show(fig)

# # Set up layouts and add to document
# inputs = column(slider)

# show(row(inputs, fig, width=800))